In [1]:
from fastapi import FastAPI, WebSocket
from track_4 import track_data, country_balls_amount
import asyncio
import glob
import numpy as np
import pandas as pd

import random
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

from fastapi_server import tracker_soft
from julia_accuracy import julia_accuracy
from fastapi_server import dist, tracker_strong, tracker_soft

Started


In [2]:
imgs = glob.glob('imgs/*')
country_balls = [{'cb_id': x, 'img': imgs[x % len(imgs)]} for x in range(country_balls_amount)]

In [3]:
def create_track(tracks_amount, random_range, bb_skip_percent):
    width = 1000
    height = 800
    tracks = []
    i = 0
    cb_width = 120
    cb_height = 100
    
    def get_point_on_random_side(width, height):
        side = random.randint(0, 4)
        if side == 0:
            x = random.randint(0, width)
            y = 0
        elif side == 1:
            x = random.randint(0, width)
            y = height
        elif side == 2:
            x = 0
            y = random.randint(0, height)
        else:
            x = width
            y = random.randint(0, height)
        return x, y

    def fun(x, a, b, c, d):
        return a * x + b * x ** 2 + c * x ** 3 + d

    def objective(x, a, b, c, d, e, f):
        return (a * x) + (b * x ** 2) + (c * x ** 3) + (d * x ** 4) + (e * x ** 5) + f

    def check_track(track):
        if all(el['x'] == track[0]['x'] for el in track):
            return False
        if all(el['y'] == track[0]['y'] for el in track):
            return False
        if not all(el['x'] >= 0 and el['x'] <= width for el in track):
            return False
        if not all(el['y'] >= 0 and el['y'] <= height for el in track):
            return False
        if (2 > track[0]['x'] > (width - 2) and 2 > track[0]['y'] > (width - 2)) or (2 > track[-1]['x'] > (width - 2) and 2 > track[-1]['y'] > (width - 2)):
            return False
        return True

    def add_track_to_tracks(track, tracks, id):
        for i, p in enumerate(track):
            # a chance that detector missed object
            if random.random() < bb_skip_percent:
                bounding_box = []
            else:
                bounding_box = [
                                  p['x'] - int(cb_width/2) + random.randint(-random_range, random_range),
                                  p['y'] - cb_height + random.randint(-random_range, random_range),
                                  p['x'] + int(cb_width/2) + random.randint(-random_range, random_range),
                                  p['y'] + random.randint(-random_range, random_range)
                                ]
            if i < len(tracks):
                tracks[i]['data'].append({'cb_id': id, 'bounding_box': bounding_box,
                                          'x': p['x'], 'y': p['y'], 'track_id': None})
            else:
                tracks.append(
                    {
                        'frame_id': len(tracks)+1,
                        'data': [{'cb_id': id, 'bounding_box': bounding_box,
                                  'x': p['x'], 'y': p['y'], 'track_id': None}]
                    }
                )
        return tracks
    
    


    while i < tracks_amount:
        x, y = np.array([]), np.array([])
        p = get_point_on_random_side(width, height)
        x = np.append(x, p[0])
        y = np.append(y, p[1])
        x = np.append(x, random.randint(200, width - 200))
        y = np.append(y, random.randint(200, height - 200))
        x = np.append(x, random.randint(200, width - 200))
        y = np.append(y, random.randint(200, height - 200))
        p = get_point_on_random_side(width, height)
        x = np.append(x, p[0])
        y = np.append(y, p[1])
        num = random.randint(20, 50)

        coef, _ = curve_fit(fun, x, y)
        track = [{'x': int(x), 'y': int(y)} for x, y in
                 zip(np.linspace(x[0], x[-1], num=num), fun(np.linspace(x[0], x[-1], num=num), *coef))]

        if check_track(track):
            tracks = add_track_to_tracks(track, tracks, i)
            i += 1
            
    return tracks

In [4]:
def test_trackers(tracks_amount=10, random_range=10, bb_skip_percent=.25):
    track = create_track(tracks_amount, random_range, bb_skip_percent)
    created_track_soft = []
    new_obj = 0
    last_frame, last_frame_2, last_frame_3 = None, None, None

    for el in track:
        el, new_obj, last_frame = tracker_soft(el, new_obj, last_frame)
        created_track_soft.append(el)

    track = create_track(tracks_amount, random_range, bb_skip_percent)
    create_track_strong = []
    new_obj = 0
    last_frame, last_frame_2, last_frame_3 = None, None, None
    for el in track:
        el, new_obj, last_frame, last_frame_2, last_frame_3 = tracker_strong(el, new_obj, last_frame, last_frame_2, last_frame_3)
        create_track_strong.append(el)

    return julia_accuracy(created_track_soft), julia_accuracy(create_track_strong)

In [5]:
report = []
for tracks_amount in [5, 10, 20]:
    for random_range in [5, 10, 20]:
        for bb_skip_percent in [.15, .25, .35]:
            soft_acc, strong_acc = test_trackers(tracks_amount, random_range, bb_skip_percent)
            report.append([tracks_amount, random_range, bb_skip_percent, soft_acc, strong_acc])
report_df = pd.DataFrame(report, columns=["tracks_amount", "random_range", "bb_skip_percent", "soft_acc", "strong_acc"])

/home/germa/miniconda3/envs/ota/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [6]:
report_df

,tracks_amount,random_range,bb_skip_percent,soft_acc,strong_acc
0,5,5,0.15,2.40,1.80
1,5,5,0.25,5.00,1.80
2,5,5,0.35,5.20,1.80
3,5,10,0.15,8.80,1.80
4,5,10,0.25,3.00,1.80
5,5,10,0.35,4.20,1.80
6,5,20,0.15,7.60,1.80
7,5,20,0.25,4.20,1.80
8,5,20,0.35,4.20,1.80
9,10,5,0.15,6.40,1.90


In [7]:
print("| " + " | ".join([i for i in report_df.columns]) + " |")
print("| " + " | ".join([":---:" for i in report_df.columns]) + " |")
for row in report_df.values:
    print("| " + " | ".join([str(i)for i in row]) + " |")

| tracks_amount | random_range | bb_skip_percent | soft_acc | strong_acc |
| :---: | :---: | :---: | :---: | :---: |
| 5.0 | 5.0 | 0.15 | 2.4 | 1.8 |
| 5.0 | 5.0 | 0.25 | 5.0 | 1.8 |
| 5.0 | 5.0 | 0.35 | 5.2 | 1.8 |
| 5.0 | 10.0 | 0.15 | 8.8 | 1.8 |
| 5.0 | 10.0 | 0.25 | 3.0 | 1.8 |
| 5.0 | 10.0 | 0.35 | 4.2 | 1.8 |
| 5.0 | 20.0 | 0.15 | 7.6 | 1.8 |
| 5.0 | 20.0 | 0.25 | 4.2 | 1.8 |
| 5.0 | 20.0 | 0.35 | 4.2 | 1.8 |
| 10.0 | 5.0 | 0.15 | 6.4 | 1.9 |
| 10.0 | 5.0 | 0.25 | 7.9 | 1.9 |
| 10.0 | 5.0 | 0.35 | 8.9 | 1.9 |
| 10.0 | 10.0 | 0.15 | 6.5 | 1.9 |
| 10.0 | 10.0 | 0.25 | 6.4 | 1.9 |
| 10.0 | 10.0 | 0.35 | 6.5 | 1.9 |
| 10.0 | 20.0 | 0.15 | 7.4 | 1.9 |
| 10.0 | 20.0 | 0.25 | 9.8 | 1.9 |
| 10.0 | 20.0 | 0.35 | 4.9 | 1.9 |
| 20.0 | 5.0 | 0.15 | 9.35 | 1.95 |
| 20.0 | 5.0 | 0.25 | 8.7 | 1.95 |
| 20.0 | 5.0 | 0.35 | 8.75 | 1.95 |
| 20.0 | 10.0 | 0.15 | 9.8 | 1.95 |
| 20.0 | 10.0 | 0.25 | 7.4 | 1.95 |
| 20.0 | 10.0 | 0.35 | 8.85 | 1.95 |
| 20.0 | 20.0 | 0.15 | 7.75 | 1.95 |
| 20.0 | 20.0 |